## Imports

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as func
from pyspark.sql.types import StringType, IntegerType
import pandas
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

## Starting spark session

In [3]:
spark = SparkSession.builder.config("spark.executor.memory", "4g").appName('House_analysis').getOrCreate()#.master('spark://192.168.1.85:7077')
spark

23/05/30 15:56:43 WARN Utils: Your hostname, DESKTOP-21T818B resolves to a loopback address: 127.0.1.1; using 172.18.5.97 instead (on interface eth0)
23/05/30 15:56:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/30 15:56:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/30 15:56:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Data load

In [6]:
df_train = spark.read.csv("./data/train.csv", header=True, inferSchema=True)
#spark_env = os.getenv('SPARK_HOME')
#df_train = spark.read.csv(f"{spark_env}/data/kaggle_data/train.csv", header=True, inferSchema=True)
df_train.toPandas()

23/05/30 15:57:09 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,2,2008,WD,Normal,208500
1,2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,5,2007,WD,Normal,181500
2,3,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,9,2008,WD,Normal,223500
3,4,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62,7917,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85,13175,Pave,NA,Reg,Lvl,AllPub,...,0,NA,MnPrv,NA,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66,9042,Pave,NA,Reg,Lvl,AllPub,...,0,NA,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68,9717,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,4,2010,WD,Normal,142125


In [7]:
#df_test = spark.read.csv(f"{spark_env}/data/kaggle_data/test.csv", header=True, inferSchema=True)
df_test = spark.read.csv("./data/test.csv", header=True, inferSchema=True)
df_test.toPandas()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80,11622,Pave,NA,Reg,Lvl,AllPub,...,120,0,NA,MnPrv,NA,0,6,2010,WD,Normal
1,1462,20,RL,81,14267,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,NA,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74,13830,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,MnPrv,NA,0,3,2010,WD,Normal
3,1464,60,RL,78,9978,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,NA,NA,0,6,2010,WD,Normal
4,1465,120,RL,43,5005,Pave,NA,IR1,HLS,AllPub,...,144,0,NA,NA,NA,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21,1936,Pave,NA,Reg,Lvl,AllPub,...,0,0,NA,NA,NA,0,6,2006,WD,Normal
1455,2916,160,RM,21,1894,Pave,NA,Reg,Lvl,AllPub,...,0,0,NA,NA,NA,0,4,2006,WD,Abnorml
1456,2917,20,RL,160,20000,Pave,NA,Reg,Lvl,AllPub,...,0,0,NA,NA,NA,0,9,2006,WD,Abnorml
1457,2918,85,RL,62,10441,Pave,NA,Reg,Lvl,AllPub,...,0,0,NA,MnPrv,Shed,700,7,2006,WD,Normal


In [8]:
df_test = df_test.withColumn('SalePrice', func.lit(None))
df_test.toPandas()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,20,RH,80,11622,Pave,NA,Reg,Lvl,AllPub,...,0,NA,MnPrv,NA,0,6,2010,WD,Normal,None
1,1462,20,RL,81,14267,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,Gar2,12500,6,2010,WD,Normal,None
2,1463,60,RL,74,13830,Pave,NA,IR1,Lvl,AllPub,...,0,NA,MnPrv,NA,0,3,2010,WD,Normal,None
3,1464,60,RL,78,9978,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,6,2010,WD,Normal,None
4,1465,120,RL,43,5005,Pave,NA,IR1,HLS,AllPub,...,0,NA,NA,NA,0,1,2010,WD,Normal,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21,1936,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,6,2006,WD,Normal,None
1455,2916,160,RM,21,1894,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,4,2006,WD,Abnorml,None
1456,2917,20,RL,160,20000,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,9,2006,WD,Abnorml,None
1457,2918,85,RL,62,10441,Pave,NA,Reg,Lvl,AllPub,...,0,NA,MnPrv,Shed,700,7,2006,WD,Normal,None


In [9]:
df = df_train.union(df_test)
df.toPandas()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2915,160,RM,21,1936,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,6,2006,WD,Normal,NaN
2915,2916,160,RM,21,1894,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,4,2006,WD,Abnorml,NaN
2916,2917,20,RL,160,20000,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,9,2006,WD,Abnorml,NaN
2917,2918,85,RL,62,10441,Pave,NA,Reg,Lvl,AllPub,...,0,NA,MnPrv,Shed,700,7,2006,WD,Normal,NaN


In [10]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

In [11]:
df.select([func.count(func.when(df[c].isNull() | func.isnan(df[c]), c)).alias(c) for c in df.columns]).show(vertical=True)

-RECORD 0-------------
 Id            | 0    
 MSSubClass    | 0    
 MSZoning      | 0    
 LotFrontage   | 0    
 LotArea       | 0    
 Street        | 0    
 Alley         | 0    
 LotShape      | 0    
 LandContour   | 0    
 Utilities     | 0    
 LotConfig     | 0    
 LandSlope     | 0    
 Neighborhood  | 0    
 Condition1    | 0    
 Condition2    | 0    
 BldgType      | 0    
 HouseStyle    | 0    
 OverallQual   | 0    
 OverallCond   | 0    
 YearBuilt     | 0    
 YearRemodAdd  | 0    
 RoofStyle     | 0    
 RoofMatl      | 0    
 Exterior1st   | 0    
 Exterior2nd   | 0    
 MasVnrType    | 0    
 MasVnrArea    | 0    
 ExterQual     | 0    
 ExterCond     | 0    
 Foundation    | 0    
 BsmtQual      | 0    
 BsmtCond      | 0    
 BsmtExposure  | 0    
 BsmtFinType1  | 0    
 BsmtFinSF1    | 0    
 BsmtFinType2  | 0    
 BsmtFinSF2    | 0    
 BsmtUnfSF     | 0    
 TotalBsmtSF   | 0    
 Heating       | 0    
 HeatingQC     | 0    
 CentralAir    | 0    
 Electrical

## Pre-processing

### Changing awful names of columns and transforming the data

#### Changing MSSubClass column

In [12]:
col = 'MSSubClass'
dict_map = {20: '1-STORY 1946 & NEWER ALL STYLES', 30: '1-STORY 1945 & OLDER', 40: '1-STORY W/FINISHED ATTIC ALL AGES', 45: '1-1/2 STORY - UNFINISHED ALL AGES',
 50: '1-1/2 STORY FINISHED ALL AGES', 60: '2-STORY 1946 & NEWER', 70: '2-STORY 1945 & OLDER', 75: '2-1/2 STORY ALL AGES', 80: 'SPLIT OR MULTI-LEVEL', 
 85: 'SPLIT FOYER', 90: 'DUPLEX - ALL STYLES AND AGES', 120: '1-STORY PUD (Planned Unit Development) - 1946 & NEWER', 150: '1-1/2 STORY PUD - ALL AGES',
 160: '2-STORY PUD - 1946 & NEWER', 180: 'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER', 190: '2 FAMILY CONVERSION - ALL STYLES AND AGES'}

user_func =  func.udf(lambda x: dict_map.get(x))#, StringType()
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'DwellingType')

#### Changing MSZoning column

In [13]:
col = 'MSZoning'
df.select(col).distinct().show()

+--------+
|MSZoning|
+--------+
| C (all)|
|      RH|
|      FV|
|      RL|
|      RM|
|      NA|
+--------+



In [14]:
dict_map = {'A': 'Agriculture', 'C (all)': 'Commercial', 'FV': 'Floating Village Residential', 'I': 'Industrial', 'RH': 'Residential High Density',
 'RL': 'Residential Low Density', 'RP': 'Residential Low Density Park', 'RM': 'Residential Medium Density', 'NA': 'NA'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'ZoneType')

#### Changing Street column

In [15]:
col = 'Street'
dict_map = {'Pave': 'Paved', 'Grvl': 'Grovel'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col]))

#### Changing Alley column

In [16]:
col = 'Alley'
dict_map = {'Pave': 'Paved', 'Grvl': 'Grovel', 'NA': 'Not Alley Access'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col]))

#### Changing LotShape column

In [17]:
col = 'LotShape'
dict_map = {'Reg': 'Regular', 'IR1': 'Slightly irregular', 'IR2': 'Moderately Irregular', 'IR3': 'Irregular'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col]))

#### Changing LandContour column

In [18]:
col = 'LandContour'
dict_map = {'Lvl': 'Near Flat/Level', 'Bnk': 'Banked', 'HLS': 'Hillside', 'Low': 'Depression'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'Flatness')

#### Changing Utilities column

In [19]:
col = 'Utilities'
df[[col]].distinct().show()

+---------+
|Utilities|
+---------+
|   NoSeWa|
|   AllPub|
|       NA|
+---------+



In [20]:
col = 'Utilities'
dict_map = {'AllPub': 'All public Utilities', 'NoSewr': 'Electricity, Gas and Water', 'NoSeWa': 'Electricity and Gas Only', 'ELO': 'Electricity Only', 'NA': 'NA'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col]))

#### Changing LandSlope column

In [21]:
col = 'LandSlope'
dict_map = {'Gtl': 'Gentle', 'Mod': 'Moderate', 'Sev': 'Severe'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col]))

#### Changing BldgType column

In [22]:
col = 'BldgType'
df.select(col).distinct().show()

+--------+
|BldgType|
+--------+
|   Twnhs|
|    1Fam|
|  Duplex|
|  2fmCon|
|  TwnhsE|
+--------+



In [23]:
dict_map = {'1Fam': 'Single-family Detached', '2fmCon': 'Two-family Conversion', 'Duplex': 'Duplex', 'TwnhsE': 'Townhouse End Unit', 'Twnhs': 'Townhouse Inside Unit'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'BuildingType')

#### Changing RoofMatl column

In [24]:
col = 'RoofMatl'
dict_map = {'ClyTile': 'Single-family Detached', 'CompShg': 'Two-family Conversion', 'Membran': 'Duplex', 'Metal': 'Metal', 'Roll': 'Roll', 'Tar&Grv': 'Gravel & Tar',
            'WdShake': 'Wood Shake', 'WdShngl': 'Wood Shingles'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'RoofMaterial')

#### Changing Exterior1st column

In [25]:
col = 'Exterior1st'
df.select(col).distinct().show()

+-----------+
|Exterior1st|
+-----------+
|    HdBoard|
|    CemntBd|
|    AsphShn|
|    WdShing|
|    Plywood|
|    ImStucc|
|    Wd Sdng|
|     Stucco|
|    MetalSd|
|      Stone|
|    AsbShng|
|    BrkComm|
|    BrkFace|
|     CBlock|
|    VinylSd|
|         NA|
+-----------+



In [26]:
dict_map = {'AsbShng': 'Asbestos Shingles', 'AsphShn': 'Asphalt Shingles', 'BrkComm': 'Brick Common', 'BrkFace': 'Brick Face', 'CBlock': 'Cinder Block', 'CemntBd': 'Cement Board',
            'HdBoard': 'Hard Board', 'ImStucc': 'Imitation Stucco', 'MetalSd': 'Metal Siding', 'Other': 'Other', 'Plywood': 'Plywood', 'PreCast': 'PreCast', 'Stone': 'Stone', 
            'Stucco': 'Stucco', 'VinylSd': 'Vinyl Siding', 'Wd Sdng': 'Wood Siding', 'WdShing': 'Wood Shingles', 'NA': 'NA'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col]))

#### Changing Exterior2nd column

In [27]:
col = 'Exterior2nd'
df.select(col).distinct().show()

+-----------+
|Exterior2nd|
+-----------+
|    CmentBd|
|    HdBoard|
|    AsphShn|
|      Other|
|    Plywood|
|    Wd Shng|
|    ImStucc|
|    Wd Sdng|
|     Stucco|
|    MetalSd|
|      Stone|
|    AsbShng|
|    Brk Cmn|
|    BrkFace|
|     CBlock|
|    VinylSd|
|         NA|
+-----------+



In [28]:
col = 'Exterior2nd'
dict_map = {'AsbShng': 'Asbestos Shingles', 'AsphShn': 'Asphalt Shingles', 'Brk Cmn': 'Brick Common', 'BrkFace': 'Brick Face', 'CBlock': 'Cinder Block', 'CmentBd': 'Cement Board',
            'HdBoard': 'Hard Board', 'ImStucc': 'Imitation Stucco', 'MetalSd': 'Metal Siding', 'Other': 'Other', 'Plywood': 'Plywood', 'PreCast': 'PreCast', 'Stone': 'Stone', 
            'Stucco': 'Stucco', 'VinylSd': 'Vinyl Siding', 'Wd Sdng': 'Wood Siding', 'Wd Shng': 'Wood Shingles', 'NA': 'NA'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col]))

#### Changing MasVnrType column

In [29]:
col = 'MasVnrType'
df.select(col).distinct().show()

+----------+
|MasVnrType|
+----------+
|      None|
|    BrkCmn|
|        NA|
|     Stone|
|   BrkFace|
+----------+



In [30]:
dict_map = {'BrkCmn': 'Brick Common', 'BrkFace': 'Brick Face', 'CBlock': 'Cinder Block', 'None': 'None', 'Stone': 'Stone', 'NA': 'NA'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'MasonryVeneerType')

#### Changing ExterQual column

In [31]:
col = 'ExterQual'
dict_map = {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}

user_func =  func.udf(lambda x: dict_map.get(x), IntegerType())
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'ExteriorQuality')

#### Changing ExterCond column

In [32]:
col = 'ExterCond'
dict_map = {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}

user_func =  func.udf(lambda x: dict_map.get(x), IntegerType())
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'ExteriorCondition')

#### Changing Foundation column

In [33]:
col = 'Foundation'
dict_map = {'BrkTil': 'Brick & Tile', 'PConc': 'Poured Contrete', 'CBlock': 'Cinder Block', 'Slab': 'Slab', 'Stone': 'Stone', 'Wood': 'Wood'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col]))

#### Changing BsmtQual column

In [34]:
col = 'BsmtQual'
dict_map = {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}

user_func =  func.udf(lambda x: dict_map.get(x), IntegerType())
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'BasementQuality')

#### Changing BsmtCond column

In [35]:
col = 'BsmtCond'
dict_map = {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}

user_func =  func.udf(lambda x: dict_map.get(x), IntegerType())
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'BasementCondition')

#### Changing BsmtExposure column

In [36]:
col = 'BsmtExposure'
df.select(col).distinct().show()

+------------+
|BsmtExposure|
+------------+
|          Gd|
|          NA|
|          Mn|
|          No|
|          Av|
+------------+



In [37]:
dict_map = {'NA': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4}

user_func =  func.udf(lambda x: dict_map.get(x), IntegerType())
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'BasementExposure')

#### Changing BsmtFinType1 column

In [38]:
col = 'BsmtFinType1'
dict_map = {'NA': 'No Basement', 'Unf': 'Unfinshed', 'LwQ': 'Low Quality', 'BLQ': 'Below Average Living Quarters', 'Rec': 'Average Rec Room', 'ALQ': 'Average Living Quarters', 'GLQ': 'Good Living Quarters'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'BasementFinishedType1')

#### Changing BsmtFinType2 column

In [39]:
col = 'BsmtFinType2'
dict_map = {'NA': 'No Basement', 'Unf': 'Unfinshed', 'LwQ': 'Low Quality', 'BLQ': 'Below Average Living Quarters', 'Rec': 'Average Rec Room', 'ALQ': 'Average Living Quarters', 'GLQ': 'Good Living Quarters'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'BasementFinishedType2')

#### Changing CentralAir column

In [40]:
col = 'CentralAir'
dict_map = {'N': 0, 'Y': 1}

user_func =  func.udf(lambda x: dict_map.get(x), IntegerType())
df = df.withColumn(col, user_func(df[col]))

#### Changing HeatingQC column

In [41]:
col = 'HeatingQC'
dict_map = {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}

user_func =  func.udf(lambda x: dict_map.get(x), IntegerType())
df = df.withColumn(col, user_func(df[col]))

#### Changing KitchenQual column

In [42]:
col = 'KitchenQual'
df.select(col).distinct().show()

+-----------+
|KitchenQual|
+-----------+
|         Gd|
|         Ex|
|         Fa|
|         TA|
|         NA|
+-----------+



In [43]:
dict_map = {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4, 'NA': 0}

user_func =  func.udf(lambda x: dict_map.get(x), IntegerType())
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'KitchenQuality')

#### Changing FireplaceQu column

In [44]:
col = 'FireplaceQu'
dict_map = {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}

user_func =  func.udf(lambda x: dict_map.get(x), IntegerType())
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'FireplaceQuality')

#### Changing GarageFinish column

In [45]:
col = 'GarageFinish'
dict_map = {'NA': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3}

user_func =  func.udf(lambda x: dict_map.get(x), IntegerType())
df = df.withColumn(col, user_func(df[col]))

#### Changing GarageQual column

In [46]:
col = 'GarageQual'
dict_map = {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}

user_func =  func.udf(lambda x: dict_map.get(x), IntegerType())
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'GarageQuality')

#### Changing FarageCond column

In [47]:
col = 'GarageCond'
dict_map = {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}

user_func =  func.udf(lambda x: dict_map.get(x), IntegerType())
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'GarageCondition')

#### Changing PavedDrive column

In [48]:
col = 'PavedDrive'
dict_map = {'N': 0, 'P': 1, 'Y': 2}

user_func =  func.udf(lambda x: dict_map.get(x), IntegerType())
df = df.withColumn(col, user_func(df[col]))

#### Changing PoolQC column

In [49]:
col = 'PoolQC'
dict_map = {'NA': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}

user_func =  func.udf(lambda x: dict_map.get(x), IntegerType())
df = df.withColumn(col, user_func(df[col])).withColumnRenamed(col, 'PoolQuality')

#### Changing Fence column

In [50]:
col = 'Fence'
dict_map = {'GdPrv': 'Good Privacy', 'MnPrv': 'Minimum Privacy', 'GdWo': 'Good Wood', 'MnWw': 'Minimum Wood/Wire', 'NA': 'No Fance'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col]))

#### Changing MiscFeature column

In [51]:
col = 'MiscFeature'
dict_map = {'Elev': 'Elevator', 'Gar2': '2nd Garage', 'Othr': 'Other', 'Shed': 'Shed', 'TenC': 'Tennis Court', 'NA': 'None'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col]))

#### Changing SaleCondition column

In [52]:
col = 'SaleCondition'
dict_map = {'Normal': 'Normal Sale', 'Abnorml': 'Abnormal Sale', 'AdjLand': 'Adjoining Land Purchase', 'Alloca': 'Allocation', 'Family': 'Sale between family members', 'Partial': 'Partial'}

user_func =  func.udf(lambda x: dict_map.get(x), StringType())
df = df.withColumn(col, user_func(df[col]))

#### Simply renaming

In [53]:
df = df.withColumnRenamed('OverallQual', 'OverallQuality')
df = df.withColumnRenamed('OverallCond', 'OverallConditions')
df = df.withColumnRenamed('YearRemodAdd', 'YearRemodel')
df = df.withColumnRenamed('MasVnrArea', 'MasonryVeneerArea')
df = df.withColumnRenamed('BsmtFinSF1', 'BasementFinishedType1Area')
df = df.withColumnRenamed('BsmtFinSF2', 'BasementFinishedType2Area')
df = df.withColumnRenamed('LowQualFinSF', 'LowQualityFinishedArea')
df = df.withColumnRenamed('GrLivArea', 'AboveGroundLivingArea')
df = df.withColumnRenamed('BsmtFullBath', 'BasementFullBathrooms')
df = df.withColumnRenamed('BsmtHalfBath', 'BasementHalfBathrooms')
df = df.withColumnRenamed('FullBath', 'FullBathrooms')
df = df.withColumnRenamed('HalfBath', 'HalfBathrooms')
df = df.withColumnRenamed('TotRmsAbvGrd', 'TotalRoomsAboveGround')
df = df.withColumnRenamed('3SsnPorch', 'ThreeSeasonPorchArea')

#### Changing column type

In [54]:
df = df.withColumn("GarageYrBlt", func.col("GarageYrBlt").cast("int"))
df = df.withColumn("BasementFinishedType1Area", func.col("BasementFinishedType1Area").cast("int"))
df = df.withColumn("BasementFinishedType2Area", func.col("BasementFinishedType2Area").cast("int"))
df = df.withColumn("LotFrontage", func.col("LotFrontage").cast("int"))
df = df.withColumn("MasonryVeneerArea", func.col("MasonryVeneerArea").cast("int"))
df = df.withColumn("TotalBsmtSF", func.col("TotalBsmtSF").cast("int"))
df = df.withColumn("BsmtUnfSF", func.col("BsmtUnfSF").cast("int"))
df = df.withColumn("GarageArea", func.col("GarageArea").cast("int"))
df = df.withColumn("GarageCars", func.col("GarageCars").cast("int"))
df = df.withColumn("BasementFullBathrooms", func.col("BasementFullBathrooms").cast("int"))
df = df.withColumn("BasementHalfBathrooms", func.col("BasementHalfBathrooms").cast("int"))

In [55]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- DwellingType: string (nullable = true)
 |-- ZoneType: string (nullable = true)
 |-- LotFrontage: integer (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- Flatness: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BuildingType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQuality: integer (nullable = true)
 |-- OverallConditions: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodel: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMaterial: string (nullable = true)
 |-- Exterior1st: string (nullab

In [56]:
df = df.drop('Id')

In [57]:
numeric_cols = [col for col, dtype in df.dtypes if dtype != 'string']
string_cols = [col for col, dtype in df.dtypes if dtype == 'string']

In [58]:
df[numeric_cols].describe().toPandas()

,summary,LotFrontage,LotArea,OverallQuality,OverallConditions,YearBuilt,YearRemodel,MasonryVeneerArea,ExteriorQuality,ExteriorCondition,...,OpenPorchSF,EnclosedPorch,ThreeSeasonPorchArea,ScreenPorch,PoolArea,PoolQuality,MiscVal,MoSold,YrSold,SalePrice
0,count,2433,2919,2919,2919,2919,2919,2896,2919,2919,...,2919,2919,2919,2919,2919,2919,2919,2919,2919,1460
1,mean,69.30579531442663,10168.11408016444,6.089071599862967,5.564576909900651,1971.3127783487496,1984.2644741349777,102.20131215469613,2.396711202466598,2.0856457690990067,...,47.486810551558754,23.098321342925658,2.602261048304214,16.062350119904078,2.2517985611510793,0.010277492291880781,50.82596779719082,6.213086673518328,2007.7927372387803,180921.19589041095
2,stddev,23.344904706927394,7886.996359105535,1.409947206614697,1.1131307466377236,30.291441534121137,20.89434423384076,179.33425303776386,0.5802933564635654,0.3723609711640206,...,67.57549339168973,64.24424559263244,25.188169331162975,56.18436511069397,35.66394596548709,0.18668097356572638,567.4022105501805,2.714761774157895,1.3149644889049965,79442.50288288663
3,min,21,1300,1,1,1872,1950,0,1,0,...,0,0,0,0,0,0,0,1,2006,34900
4,max,313,215245,10,9,2010,2010,1600,4,4,...,742,1012,508,576,800,4,17000,12,2010,755000


In [59]:
df.select([func.count(func.when(df[c].isNull() | func.isnan(df[c]), c)).alias(c) for c in df.columns]).show(vertical=True)

-RECORD 0-------------------------
 DwellingType              | 0    
 ZoneType                  | 0    
 LotFrontage               | 486  
 LotArea                   | 0    
 Street                    | 0    
 Alley                     | 0    
 LotShape                  | 0    
 Flatness                  | 0    
 Utilities                 | 0    
 LotConfig                 | 0    
 LandSlope                 | 0    
 Neighborhood              | 0    
 Condition1                | 0    
 Condition2                | 0    
 BuildingType              | 0    
 HouseStyle                | 0    
 OverallQuality            | 0    
 OverallConditions         | 0    
 YearBuilt                 | 0    
 YearRemodel               | 0    
 RoofStyle                 | 0    
 RoofMaterial              | 0    
 Exterior1st               | 0    
 Exterior2nd               | 0    
 MasonryVeneerType         | 0    
 MasonryVeneerArea         | 23   
 ExteriorQuality           | 0    
 ExteriorCondition  

In [60]:
df[['GarageYrBlt', 'YearBuilt', 'YearRemodel']].show()

+-----------+---------+-----------+
|GarageYrBlt|YearBuilt|YearRemodel|
+-----------+---------+-----------+
|       2003|     2003|       2003|
|       1976|     1976|       1976|
|       2001|     2001|       2002|
|       1998|     1915|       1970|
|       2000|     2000|       2000|
|       1993|     1993|       1995|
|       2004|     2004|       2005|
|       1973|     1973|       1973|
|       1931|     1931|       1950|
|       1939|     1939|       1950|
|       1965|     1965|       1965|
|       2005|     2005|       2006|
|       1962|     1962|       1962|
|       2006|     2006|       2007|
|       1960|     1960|       1960|
|       1991|     1929|       2001|
|       1970|     1970|       1970|
|       1967|     1967|       1967|
|       2004|     2004|       2004|
|       1958|     1958|       1965|
+-----------+---------+-----------+
only showing top 20 rows



In [61]:
df[df['GarageYrBlt'] > 2020][['GarageYrBlt', 'YearBuilt', 'YearRemodel']].show()

+-----------+---------+-----------+
|GarageYrBlt|YearBuilt|YearRemodel|
+-----------+---------+-----------+
|       2207|     2006|       2007|
+-----------+---------+-----------+



In [62]:
df = df.withColumn('GarageYrBlt', func.when(df['GarageYrBlt'] == 2207, 2007).otherwise(df['GarageYrBlt']))

In [63]:
df_garage = df[['GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQuality', 'GarageCondition', 'GarageType']] #ver nulls
df_garage.show()

+-----------+------------+----------+----------+-------------+---------------+----------+
|GarageYrBlt|GarageFinish|GarageCars|GarageArea|GarageQuality|GarageCondition|GarageType|
+-----------+------------+----------+----------+-------------+---------------+----------+
|       2003|           2|         2|       548|            3|              3|    Attchd|
|       1976|           2|         2|       460|            3|              3|    Attchd|
|       2001|           2|         2|       608|            3|              3|    Attchd|
|       1998|           1|         3|       642|            3|              3|    Detchd|
|       2000|           2|         3|       836|            3|              3|    Attchd|
|       1993|           1|         2|       480|            3|              3|    Attchd|
|       2004|           2|         2|       636|            3|              3|    Attchd|
|       1973|           2|         2|       484|            3|              3|    Attchd|
|       19

In [64]:
for col in df_garage.columns:
    print(col)
    df_garage[df_garage[col].isNull()].show()

GarageYrBlt
+-----------+------------+----------+----------+-------------+---------------+----------+
|GarageYrBlt|GarageFinish|GarageCars|GarageArea|GarageQuality|GarageCondition|GarageType|
+-----------+------------+----------+----------+-------------+---------------+----------+
|       null|           0|         0|         0|            0|              0|        NA|
|       null|           0|         0|         0|            0|              0|        NA|
|       null|           0|         0|         0|            0|              0|        NA|
|       null|           0|         0|         0|            0|              0|        NA|
|       null|           0|         0|         0|            0|              0|        NA|
|       null|           0|         0|         0|            0|              0|        NA|
|       null|           0|         0|         0|            0|              0|        NA|
|       null|           0|         0|         0|            0|              0|        NA

In [65]:
df = df.fillna(subset=['GarageYrBlt', 'GarageCars', 'GarageArea'], value=0)

In [66]:
df_mansonry = df[['MasonryVeneerType', 'MasonryVeneerArea']]
df_mansonry.show()

+-----------------+-----------------+
|MasonryVeneerType|MasonryVeneerArea|
+-----------------+-----------------+
|       Brick Face|              196|
|             None|                0|
|       Brick Face|              162|
|             None|                0|
|       Brick Face|              350|
|             None|                0|
|            Stone|              186|
|            Stone|              240|
|             None|                0|
|             None|                0|
|             None|                0|
|            Stone|              286|
|             None|                0|
|            Stone|              306|
|       Brick Face|              212|
|             None|                0|
|       Brick Face|              180|
|             None|                0|
|             None|                0|
|             None|                0|
+-----------------+-----------------+
only showing top 20 rows



In [67]:
df_mansonry[df_mansonry['MasonryVeneerArea'].isNull()].show()

+-----------------+-----------------+
|MasonryVeneerType|MasonryVeneerArea|
+-----------------+-----------------+
|               NA|             null|
|               NA|             null|
|               NA|             null|
|               NA|             null|
|               NA|             null|
|               NA|             null|
|               NA|             null|
|               NA|             null|
|               NA|             null|
|               NA|             null|
|               NA|             null|
|               NA|             null|
|               NA|             null|
|               NA|             null|
|               NA|             null|
|               NA|             null|
|               NA|             null|
|               NA|             null|
|               NA|             null|
|               NA|             null|
+-----------------+-----------------+
only showing top 20 rows



In [68]:
#df_mansonry[df_mansonry['MasonryVeneerArea'] == 0].show()
df_mansonry[df_mansonry['MasonryVeneerType'] == 'None'].show()#NA

+-----------------+-----------------+
|MasonryVeneerType|MasonryVeneerArea|
+-----------------+-----------------+
|             None|                0|
|             None|                0|
|             None|                0|
|             None|                0|
|             None|                0|
|             None|                0|
|             None|                0|
|             None|                0|
|             None|                0|
|             None|                0|
|             None|                0|
|             None|                0|
|             None|                0|
|             None|                0|
|             None|                0|
|             None|                0|
|             None|                0|
|             None|                0|
|             None|                0|
|             None|                0|
+-----------------+-----------------+
only showing top 20 rows



In [69]:
df = df.withColumn('MasonryVeneerType', func.when(df_mansonry['MasonryVeneerType'] == 'None', 'NA').otherwise(df_mansonry['MasonryVeneerType']))
df = df.fillna(subset='MasonryVeneerArea', value=0)

In [70]:
df_bathrooms = df[['BasementFullBathrooms', 'BasementHalfBathrooms', 'BasementQuality', 'BasementFinishedType1', 'BasementFinishedType2', 'BasementFinishedType1Area', 'BasementFinishedType2Area', 'BsmtUnfSF', 'TotalBsmtSF']]
df_bathrooms.show()

+---------------------+---------------------+---------------+---------------------+---------------------+-------------------------+-------------------------+---------+-----------+
|BasementFullBathrooms|BasementHalfBathrooms|BasementQuality|BasementFinishedType1|BasementFinishedType2|BasementFinishedType1Area|BasementFinishedType2Area|BsmtUnfSF|TotalBsmtSF|
+---------------------+---------------------+---------------+---------------------+---------------------+-------------------------+-------------------------+---------+-----------+
|                    1|                    0|              4| Good Living Quarters|            Unfinshed|                      706|                        0|      150|        856|
|                    0|                    1|              4| Average Living Qu...|            Unfinshed|                      978|                        0|      284|       1262|
|                    1|                    0|              4| Good Living Quarters|            Unfin

In [71]:
for col in df_bathrooms.columns:
    print(col)
    df_bathrooms[df_bathrooms[col].isNull()].show()

BasementFullBathrooms
+---------------------+---------------------+---------------+---------------------+---------------------+-------------------------+-------------------------+---------+-----------+
|BasementFullBathrooms|BasementHalfBathrooms|BasementQuality|BasementFinishedType1|BasementFinishedType2|BasementFinishedType1Area|BasementFinishedType2Area|BsmtUnfSF|TotalBsmtSF|
+---------------------+---------------------+---------------+---------------------+---------------------+-------------------------+-------------------------+---------+-----------+
|                 null|                 null|              0|          No Basement|          No Basement|                     null|                     null|     null|       null|
|                 null|                 null|              0|          No Basement|          No Basement|                        0|                        0|        0|          0|
+---------------------+---------------------+---------------+-----------------

In [72]:
df = df.fillna(subset=['BasementFullBathrooms', 'BasementHalfBathrooms', 'BsmtUnfSF', 'TotalBsmtSF', 'BasementFinishedType1Area', 'BasementFinishedType2Area'], value=0)

In [73]:
df_lot = df[['LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape']]
df_lot.show()

+-----------+-------+------+----------------+--------------------+
|LotFrontage|LotArea|Street|           Alley|            LotShape|
+-----------+-------+------+----------------+--------------------+
|         65|   8450| Paved|Not Alley Access|             Regular|
|         80|   9600| Paved|Not Alley Access|             Regular|
|         68|  11250| Paved|Not Alley Access|  Slightly irregular|
|         60|   9550| Paved|Not Alley Access|  Slightly irregular|
|         84|  14260| Paved|Not Alley Access|  Slightly irregular|
|         85|  14115| Paved|Not Alley Access|  Slightly irregular|
|         75|  10084| Paved|Not Alley Access|             Regular|
|       null|  10382| Paved|Not Alley Access|  Slightly irregular|
|         51|   6120| Paved|Not Alley Access|             Regular|
|         50|   7420| Paved|Not Alley Access|             Regular|
|         70|  11200| Paved|Not Alley Access|             Regular|
|         85|  11924| Paved|Not Alley Access|  Slightly irregu

In [74]:
df_lot[df_lot['LotFrontage'].isNull()].show()

+-----------+-------+------+----------------+--------------------+
|LotFrontage|LotArea|Street|           Alley|            LotShape|
+-----------+-------+------+----------------+--------------------+
|       null|  10382| Paved|Not Alley Access|  Slightly irregular|
|       null|  12968| Paved|Not Alley Access|Moderately Irregular|
|       null|  10920| Paved|Not Alley Access|  Slightly irregular|
|       null|  11241| Paved|Not Alley Access|  Slightly irregular|
|       null|   8246| Paved|Not Alley Access|  Slightly irregular|
|       null|   8544| Paved|Not Alley Access|  Slightly irregular|
|       null|   9180| Paved|Not Alley Access|  Slightly irregular|
|       null|   9200| Paved|Not Alley Access|  Slightly irregular|
|       null|  13869| Paved|Not Alley Access|Moderately Irregular|
|       null|   9375| Paved|Not Alley Access|             Regular|
|       null|  19900| Paved|Not Alley Access|             Regular|
|       null|   8475| Paved|Not Alley Access|  Slightly irregu

In [75]:
df = df.fillna(subset=['LotFrontage'], value=0)

### Feature engineering

In [76]:
df = df.withColumn('TotalUsedArea', df['AboveGroundLivingArea'] + df['GarageArea'] + df['TotalBsmtSF'])
df[['TotalUsedArea', 'AboveGroundLivingArea', 'GarageArea', 'TotalBsmtSF']].show()

+-------------+---------------------+----------+-----------+
|TotalUsedArea|AboveGroundLivingArea|GarageArea|TotalBsmtSF|
+-------------+---------------------+----------+-----------+
|         3114|                 1710|       548|        856|
|         2984|                 1262|       460|       1262|
|         3314|                 1786|       608|        920|
|         3115|                 1717|       642|        756|
|         4179|                 2198|       836|       1145|
|         2638|                 1362|       480|        796|
|         4016|                 1694|       636|       1686|
|         3681|                 2090|       484|       1107|
|         3194|                 1774|       468|        952|
|         2273|                 1077|       205|        991|
|         2464|                 1040|       384|       1040|
|         4235|                 2324|       736|       1175|
|         2176|                  912|       352|        912|
|         3828|         

In [77]:
df = df.withColumn('NonbuiltArea', df['LotArea'] - (df['AboveGroundLivingArea'] + df['GarageArea'] - df['2ndFlrSF']))
df = df.withColumn('BuiltAreaRatio', df['AboveGroundLivingArea'] / df['LotArea'])
df[['NonbuiltArea', 'BuiltAreaRatio']].show()

+------------+-------------------+
|NonbuiltArea|     BuiltAreaRatio|
+------------+-------------------+
|        7046|0.20236686390532543|
|        7878|0.13145833333333334|
|        9722|0.15875555555555557|
|        7947|0.17979057591623038|
|       12279| 0.1541374474053296|
|       12839|0.09649309245483528|
|        7754|0.16798889329631098|
|        8791|0.20130995954536698|
|        4630|0.28986928104575166|
|        6138|0.14514824797843665|
|        9776|0.09285714285714286|
|       10006| 0.1949010399194901|
|       11704| 0.0703269586674892|
|        8318|0.14025535110777318|
|        9315|0.11474358974358974|
|        4690|0.13954248366013072|
|        9757|0.08931589716217418|
|        8979|0.12010008340283569|
|       12005|0.08134355604235122|
|        5927|0.17711640211640212|
+------------+-------------------+
only showing top 20 rows



In [78]:
df = df.withColumn('PorchArea', df['OpenPorchSF'] + df['EnclosedPorch'] + df['ThreeSeasonPorchArea'] + df['ScreenPorch'])
df[['PorchArea', 'OpenPorchSF', 'EnclosedPorch', 'ThreeSeasonPorchArea', 'ScreenPorch']].show()

+---------+-----------+-------------+--------------------+-----------+
|PorchArea|OpenPorchSF|EnclosedPorch|ThreeSeasonPorchArea|ScreenPorch|
+---------+-----------+-------------+--------------------+-----------+
|       61|         61|            0|                   0|          0|
|        0|          0|            0|                   0|          0|
|       42|         42|            0|                   0|          0|
|      307|         35|          272|                   0|          0|
|       84|         84|            0|                   0|          0|
|      350|         30|            0|                 320|          0|
|       57|         57|            0|                   0|          0|
|      432|        204|          228|                   0|          0|
|      205|          0|          205|                   0|          0|
|        4|          4|            0|                   0|          0|
|        0|          0|            0|                   0|          0|
|     

In [79]:
user_func =  func.udf(lambda x: 1 if x > 0 else 0, IntegerType())
df = df.withColumn('IsDuplex', user_func(df['2ndFlrSF']))
df[['IsDuplex', '1stFlrSF', '2ndFlrSF']].show()

+--------+--------+--------+
|IsDuplex|1stFlrSF|2ndFlrSF|
+--------+--------+--------+
|       1|     856|     854|
|       0|    1262|       0|
|       1|     920|     866|
|       1|     961|     756|
|       1|    1145|    1053|
|       1|     796|     566|
|       0|    1694|       0|
|       1|    1107|     983|
|       1|    1022|     752|
|       0|    1077|       0|
|       0|    1040|       0|
|       1|    1182|    1142|
|       0|     912|       0|
|       0|    1494|       0|
|       0|    1253|       0|
|       0|     854|       0|
|       0|    1004|       0|
|       0|    1296|       0|
|       0|    1114|       0|
|       0|    1339|       0|
+--------+--------+--------+
only showing top 20 rows



In [80]:
user_func =  func.udf(lambda x, y: 1 if x < y else 0, IntegerType())
df = df.withColumn('SoldAfterRemodel', user_func(df['YearBuilt'], df['YearRemodel']))
df[['SoldAfterRemodel', 'YearBuilt', 'YearRemodel']].show()

+----------------+---------+-----------+
|SoldAfterRemodel|YearBuilt|YearRemodel|
+----------------+---------+-----------+
|               0|     2003|       2003|
|               0|     1976|       1976|
|               1|     2001|       2002|
|               1|     1915|       1970|
|               0|     2000|       2000|
|               1|     1993|       1995|
|               1|     2004|       2005|
|               0|     1973|       1973|
|               1|     1931|       1950|
|               1|     1939|       1950|
|               0|     1965|       1965|
|               1|     2005|       2006|
|               0|     1962|       1962|
|               1|     2006|       2007|
|               0|     1960|       1960|
|               1|     1929|       2001|
|               0|     1970|       1970|
|               0|     1967|       1967|
|               0|     2004|       2004|
|               1|     1958|       1965|
+----------------+---------+-----------+
only showing top

In [81]:
df = df.withColumn('RemodelAge', df['YrSold'] - df['YearRemodel'])
df[['RemodelAge', 'YrSold', 'YearRemodel']].show()

+----------+------+-----------+
|RemodelAge|YrSold|YearRemodel|
+----------+------+-----------+
|         5|  2008|       2003|
|        31|  2007|       1976|
|         6|  2008|       2002|
|        36|  2006|       1970|
|         8|  2008|       2000|
|        14|  2009|       1995|
|         2|  2007|       2005|
|        36|  2009|       1973|
|        58|  2008|       1950|
|        58|  2008|       1950|
|        43|  2008|       1965|
|         0|  2006|       2006|
|        46|  2008|       1962|
|         0|  2007|       2007|
|        48|  2008|       1960|
|         6|  2007|       2001|
|        40|  2010|       1970|
|        39|  2006|       1967|
|         4|  2008|       2004|
|        44|  2009|       1965|
+----------+------+-----------+
only showing top 20 rows



In [82]:
df = df.withColumn('FireplacesLivingAreaRatio', df['Fireplaces'] / df['AboveGroundLivingArea'])
df[['FireplacesLivingAreaRatio', 'Fireplaces', 'AboveGroundLivingArea']].show()

+-------------------------+----------+---------------------+
|FireplacesLivingAreaRatio|Fireplaces|AboveGroundLivingArea|
+-------------------------+----------+---------------------+
|                      0.0|         0|                 1710|
|      7.92393026941363E-4|         1|                 1262|
|     5.599104143337066E-4|         1|                 1786|
|     5.824111822947001E-4|         1|                 1717|
|     4.549590536851683...|         1|                 2198|
|                      0.0|         0|                 1362|
|     5.903187721369539E-4|         1|                 1694|
|     9.569377990430622E-4|         2|                 2090|
|     0.001127395715896...|         2|                 1774|
|     0.001857010213556...|         2|                 1077|
|                      0.0|         0|                 1040|
|     8.605851979345956E-4|         2|                 2324|
|                      0.0|         0|                  912|
|     6.693440428380187E

In [83]:
df = df.withColumn('BedroomsBathroomsRatio', df['BedroomAbvGr'] / (df['FullBathrooms'] + df['HalfBathrooms']))
df = df.fillna(subset='BedroomsBathroomsRatio', value=0)
df[['BedroomsBathroomsRatio', 'BedroomAbvGr', 'FullBathrooms', 'HalfBathrooms']].show()

+----------------------+------------+-------------+-------------+
|BedroomsBathroomsRatio|BedroomAbvGr|FullBathrooms|HalfBathrooms|
+----------------------+------------+-------------+-------------+
|                   1.0|           3|            2|            1|
|                   1.5|           3|            2|            0|
|                   1.0|           3|            2|            1|
|                   3.0|           3|            1|            0|
|    1.3333333333333333|           4|            2|            1|
|                   0.5|           1|            1|            1|
|                   1.5|           3|            2|            0|
|                   1.0|           3|            2|            1|
|                   1.0|           2|            2|            0|
|                   2.0|           2|            1|            0|
|                   3.0|           3|            1|            0|
|    1.3333333333333333|           4|            3|            0|
|         

In [84]:
df = df.withColumn('HouseAge', df['YrSold'] - df['YearBuilt'])
df[['HouseAge', 'YrSold', 'YearBuilt']].show()

+--------+------+---------+
|HouseAge|YrSold|YearBuilt|
+--------+------+---------+
|       5|  2008|     2003|
|      31|  2007|     1976|
|       7|  2008|     2001|
|      91|  2006|     1915|
|       8|  2008|     2000|
|      16|  2009|     1993|
|       3|  2007|     2004|
|      36|  2009|     1973|
|      77|  2008|     1931|
|      69|  2008|     1939|
|      43|  2008|     1965|
|       1|  2006|     2005|
|      46|  2008|     1962|
|       1|  2007|     2006|
|      48|  2008|     1960|
|      78|  2007|     1929|
|      40|  2010|     1970|
|      39|  2006|     1967|
|       4|  2008|     2004|
|      51|  2009|     1958|
+--------+------+---------+
only showing top 20 rows



In [85]:
df = df.withColumn('PercentFrontage', df['LotFrontage'] / df['LotArea'])
df[['PercentFrontage', 'LotFrontage', 'LotArea']].show()

+--------------------+-----------+-------+
|     PercentFrontage|LotFrontage|LotArea|
+--------------------+-----------+-------+
|0.007692307692307693|         65|   8450|
|0.008333333333333333|         80|   9600|
|0.006044444444444444|         68|  11250|
|0.006282722513089...|         60|   9550|
|0.005890603085553997|         84|  14260|
|0.006021962451292...|         85|  14115|
|0.007437524791749306|         75|  10084|
|                 0.0|          0|  10382|
|0.008333333333333333|         51|   6120|
|0.006738544474393531|         50|   7420|
|             0.00625|         70|  11200|
|0.007128480375712848|         85|  11924|
|                 0.0|          0|  12968|
|0.008542996620352985|         91|  10652|
|                 0.0|          0|  10920|
|0.008333333333333333|         51|   6120|
|                 0.0|          0|  11241|
|0.006672226855713094|         72|  10791|
|0.004819277108433735|         66|  13695|
|0.009259259259259259|         70|   7560|
+----------

In [86]:
df = df.withColumn('ImpactView', df['PercentFrontage'] * df['OverallConditions'])
df[['ImpactView', 'PercentFrontage', 'OverallConditions']].show()

+--------------------+--------------------+-----------------+
|          ImpactView|     PercentFrontage|OverallConditions|
+--------------------+--------------------+-----------------+
|0.038461538461538464|0.007692307692307693|                5|
| 0.06666666666666667|0.008333333333333333|                8|
| 0.03022222222222222|0.006044444444444444|                5|
|0.031413612565445025|0.006282722513089...|                5|
|0.029453015427769985|0.005890603085553997|                5|
|0.030109812256464752|0.006021962451292...|                5|
| 0.03718762395874653|0.007437524791749306|                5|
|                 0.0|                 0.0|                6|
|0.041666666666666664|0.008333333333333333|                5|
| 0.04043126684636119|0.006738544474393531|                6|
|             0.03125|             0.00625|                5|
| 0.03564240187856424|0.007128480375712848|                5|
|                 0.0|                 0.0|                6|
| 0.0427

In [87]:
df = df.withColumn('RoomsMeanSize', df['AboveGroundLivingArea'] / df['TotalRoomsAboveGround'])
df[['RoomsMeanSize', 'TotalRoomsAboveGround', 'AboveGroundLivingArea']].show()

+------------------+---------------------+---------------------+
|     RoomsMeanSize|TotalRoomsAboveGround|AboveGroundLivingArea|
+------------------+---------------------+---------------------+
|            213.75|                    8|                 1710|
|210.33333333333334|                    6|                 1262|
| 297.6666666666667|                    6|                 1786|
|245.28571428571428|                    7|                 1717|
|244.22222222222223|                    9|                 2198|
|             272.4|                    5|                 1362|
|             242.0|                    7|                 1694|
|298.57142857142856|                    7|                 2090|
|            221.75|                    8|                 1774|
|             215.4|                    5|                 1077|
|             208.0|                    5|                 1040|
|211.27272727272728|                   11|                 2324|
|             228.0|     

In [88]:
df = df.withColumn('RegularRooms', df['TotalRoomsAboveGround'] - df['KitchenAbvGr'] - df['BedroomAbvGr'])
df[['RegularRooms', 'TotalRoomsAboveGround', 'KitchenAbvGr', 'BedroomAbvGr']].show()

+------------+---------------------+------------+------------+
|RegularRooms|TotalRoomsAboveGround|KitchenAbvGr|BedroomAbvGr|
+------------+---------------------+------------+------------+
|           4|                    8|           1|           3|
|           2|                    6|           1|           3|
|           2|                    6|           1|           3|
|           3|                    7|           1|           3|
|           4|                    9|           1|           4|
|           3|                    5|           1|           1|
|           3|                    7|           1|           3|
|           3|                    7|           1|           3|
|           4|                    8|           2|           2|
|           1|                    5|           2|           2|
|           1|                    5|           1|           3|
|           6|                   11|           1|           4|
|           1|                    4|           1|      

In [89]:
df = df.withColumn('GarageAreaPerCars', df['GarageArea'] / df['GarageCars'])
df = df.fillna(subset='GarageAreaPerCars', value=0)
df[['GarageAreaPerCars', 'GarageArea', 'GarageCars']].show()

+------------------+----------+----------+
| GarageAreaPerCars|GarageArea|GarageCars|
+------------------+----------+----------+
|             274.0|       548|         2|
|             230.0|       460|         2|
|             304.0|       608|         2|
|             214.0|       642|         3|
| 278.6666666666667|       836|         3|
|             240.0|       480|         2|
|             318.0|       636|         2|
|             242.0|       484|         2|
|             234.0|       468|         2|
|             205.0|       205|         1|
|             384.0|       384|         1|
|245.33333333333334|       736|         3|
|             352.0|       352|         1|
|             280.0|       840|         3|
|             352.0|       352|         1|
|             288.0|       576|         2|
|             240.0|       480|         2|
|             258.0|       516|         2|
|             288.0|       576|         2|
|             294.0|       294|         1|
+----------

In [90]:
df = df.withColumn('GarageAreaQuality', df['GarageArea'] * df['GarageQuality'])
df[['GarageAreaQuality', 'GarageArea', 'GarageQuality']].show()

+-----------------+----------+-------------+
|GarageAreaQuality|GarageArea|GarageQuality|
+-----------------+----------+-------------+
|             1644|       548|            3|
|             1380|       460|            3|
|             1824|       608|            3|
|             1926|       642|            3|
|             2508|       836|            3|
|             1440|       480|            3|
|             1908|       636|            3|
|             1452|       484|            3|
|              936|       468|            2|
|              820|       205|            4|
|             1152|       384|            3|
|             2208|       736|            3|
|             1056|       352|            3|
|             2520|       840|            3|
|             1056|       352|            3|
|             1728|       576|            3|
|             1440|       480|            3|
|             1548|       516|            3|
|             1728|       576|            3|
|         

In [91]:
df = df.withColumn('BasementAreaQuality', df['TotalBsmtSF'] * df['BasementQuality'])
df[['BasementAreaQuality', 'TotalBsmtSF', 'BasementQuality']].show()

+-------------------+-----------+---------------+
|BasementAreaQuality|TotalBsmtSF|BasementQuality|
+-------------------+-----------+---------------+
|               3424|        856|              4|
|               5048|       1262|              4|
|               3680|        920|              4|
|               2268|        756|              3|
|               4580|       1145|              4|
|               3184|        796|              4|
|               8430|       1686|              5|
|               4428|       1107|              4|
|               2856|        952|              3|
|               2973|        991|              3|
|               3120|       1040|              3|
|               5875|       1175|              5|
|               2736|        912|              3|
|               5976|       1494|              4|
|               3759|       1253|              3|
|               2496|        832|              3|
|               3012|       1004|              3|


In [92]:
df = df.withColumn('LivingAreaQuality', df['AboveGroundLivingArea'] * df['OverallQuality'])
df[['LivingAreaQuality', 'AboveGroundLivingArea', 'OverallQuality']].show()

+-----------------+---------------------+--------------+
|LivingAreaQuality|AboveGroundLivingArea|OverallQuality|
+-----------------+---------------------+--------------+
|            11970|                 1710|             7|
|             7572|                 1262|             6|
|            12502|                 1786|             7|
|            12019|                 1717|             7|
|            17584|                 2198|             8|
|             6810|                 1362|             5|
|            13552|                 1694|             8|
|            14630|                 2090|             7|
|            12418|                 1774|             7|
|             5385|                 1077|             5|
|             5200|                 1040|             5|
|            20916|                 2324|             9|
|             4560|                  912|             5|
|            10458|                 1494|             7|
|             7518|            

In [93]:
df = df.withColumn('LivingAreaCondition', df['AboveGroundLivingArea'] * df['OverallConditions'])
df[['LivingAreaCondition', 'AboveGroundLivingArea', 'OverallConditions']].show()

+-------------------+---------------------+-----------------+
|LivingAreaCondition|AboveGroundLivingArea|OverallConditions|
+-------------------+---------------------+-----------------+
|               8550|                 1710|                5|
|              10096|                 1262|                8|
|               8930|                 1786|                5|
|               8585|                 1717|                5|
|              10990|                 2198|                5|
|               6810|                 1362|                5|
|               8470|                 1694|                5|
|              12540|                 2090|                6|
|               8870|                 1774|                5|
|               6462|                 1077|                6|
|               5200|                 1040|                5|
|              11620|                 2324|                5|
|               5472|                  912|                6|
|       

In [94]:
df = df.withColumn('BasementAreaCondition', df['TotalBsmtSF'] * df['BasementCondition'])
df[['BasementAreaCondition', 'TotalBsmtSF', 'BasementCondition']].show()

+---------------------+-----------+-----------------+
|BasementAreaCondition|TotalBsmtSF|BasementCondition|
+---------------------+-----------+-----------------+
|                 2568|        856|                3|
|                 3786|       1262|                3|
|                 2760|        920|                3|
|                 3024|        756|                4|
|                 3435|       1145|                3|
|                 2388|        796|                3|
|                 5058|       1686|                3|
|                 3321|       1107|                3|
|                 2856|        952|                3|
|                 2973|        991|                3|
|                 3120|       1040|                3|
|                 3525|       1175|                3|
|                 2736|        912|                3|
|                 4482|       1494|                3|
|                 3759|       1253|                3|
|                 2496|     

In [95]:
df = df.withColumn('GarageAreaCondition', df['GarageArea'] * df['GarageCondition'])
df[['GarageAreaCondition', 'GarageArea', 'GarageCondition']].show()

+-------------------+----------+---------------+
|GarageAreaCondition|GarageArea|GarageCondition|
+-------------------+----------+---------------+
|               1644|       548|              3|
|               1380|       460|              3|
|               1824|       608|              3|
|               1926|       642|              3|
|               2508|       836|              3|
|               1440|       480|              3|
|               1908|       636|              3|
|               1452|       484|              3|
|               1404|       468|              3|
|                615|       205|              3|
|               1152|       384|              3|
|               2208|       736|              3|
|               1056|       352|              3|
|               2520|       840|              3|
|               1056|       352|              3|
|               1728|       576|              3|
|               1440|       480|              3|
|               1548

In [97]:
df.select([func.count(func.when(df[c].isNull() | func.isnan(df[c]), c)).alias(c) for c in df.columns[-17:]]).show(vertical=True)

-RECORD 0------------------------
 IsDuplex                  | 0   
 SoldAfterRemodel          | 0   
 RemodelAge                | 0   
 FireplacesLivingAreaRatio | 0   
 BedroomsBathroomsRatio    | 0   
 HouseAge                  | 0   
 PercentFrontage           | 0   
 ImpactView                | 0   
 RoomsMeanSize             | 0   
 RegularRooms              | 0   
 GarageAreaPerCars         | 0   
 GarageAreaQuality         | 0   
 BasementAreaQuality       | 0   
 LivingAreaQuality         | 0   
 LivingAreaCondition       | 0   
 BasementAreaCondition     | 0   
 GarageAreaCondition       | 0   



In [98]:
df.write.csv(path='data/house_data', header=True, mode='overwrite')